# Module 4: Using DYAD to accelerate distributed Deep Learning (DL) training

Now that we have seen how Flux enables the management and deployment of services, let's look at an example of using DYAD, an advanced Flux service for runtime data movement, in a real world application. Specifically, we will show how DYAD speeds up distributed Deep Learning (DL) training. In this module, we cover these topics:
1. Design of DYAD
2. Distributed DL training
3. Deep Learning I/O (DLIO) benchmark
4. Accelerating distributed DL training

## Design of DYAD

DYAD provides transparent, locality-aware, write-once, read-many file caching that runs on top of local NVMe and other burst buffer-style technologies (e.g., El Capitan Rabbit nodes). Figure X shows the components of DYAD, including the DYAD service (implemented as a Flux broker module), the DYAD client, and DYAD's data transport layer. DYAD uses the Flux KVS to store metadata about tracked files, and it uses Flux's remote proceedure call capabilities to communicate between client and service. DYAD also uses UCX to perform RDMA-based data transfer to move files.

<figure>
<img src="img/dyad-software-stack.png" width=50% height=auto>
<figcaption>
<i>Image created by Ian Lumsden for a poster at SC'23</i></figcaption>
</figure>

DYAD is designed to accelerate large, distributed workloads, such as distributed Deep Learning (DL) training and scientific computing workflows, on HPC systems. It is also designed be transparent, which allows users to leverage DYAD with little to no code refactoring. Unlike similar tools (e.g., DataSpaces and UnifyFS), which tend to optimize for write performance, DYAD aims to provide good write **and read** performance. To optimize read performance, DYAD uses a locality-aware "Hierarchical Data Locator," which prioritizes node-local metadata and data retrieval to minimize the amount of network communications. When moving data from another node, DYAD also uses a streaming RPC over RDMA protocol, which uses preallocated buffers and connection caching to maximize network bandwidth. This process is shown in the figure below:

<figure>
<img src="img/dyad_design.png">
<figcaption>
<i>Image created by Hari Devarajan for a paper submitted to SC'24</i></figcaption>
</figure>

## Distributed DL Training

Distributed DL training is an approach to speed up the training of large Deep Learning models by performing multiple epochs of training in parallel across multiple GPUs and, oftentimes, multiple nodes. This approach is supported by most major DL libraries, such as PyTorch and Tensorflow. In this module, we focus on PyTorch. When running training across multiple nodes and GPUs, PyTorch starts by spawning one process per GPU, called the worker. Each worker performs three major tasks:
1. Determining which samples from the dataset will comprise the batch for the next epoch of training (i.e., epoch *N+1*)
2. Reading these samples from the filesystem
3. Building a batch from these samples and moving the batch to the GPU

To assist with reading the samples from the filesystem, each worker also spawns additional I/O processes. Each of these processes reads data and, optionally, transforms the data based on the configuration of the training pipeline. Figure X shows this process for a single GPU, a single worker, and a single spawned I/O process. In this figure, "I/O" indicates data being read from the filesystem, and "Map" indicates the optional transformation of data. "Batch" indicates the building of a batch from the read samples.

<figure>
<img src="img/dl-training-io.png">
<figcaption>
<i>Image created by Ian Lumsden based on an image from <a href="https://towardsdatascience.com/building-efficient-data-pipelines-using-tensorflow-8f647f03b4ce">this article</a></i></figcaption>
</figure>

One key difference between distributed DL training and many conventional HPC applications (e.g., MPI-based simulations) is the asynchronous loading of data by workers during training. In many conventional HPC applications, data loading and computation are performed one after the one. On the other hand, as shown in Figure X, the loading of data in distributed DL training is asynchronous. In other words, while the GPU is training the DL model for epoch *N*, the worker reading and creating the batch for epoch *N+1*. This asynchronous loading of data can lead to imbalance between data loading and training. For example, Figure X shows a scenario where the data loading takes longer than training, resulting in idle time on the GPU, wasted resources, and, overall, an I/O bound application.

At the end of each epoch of training, all workers and GPUs are synchronized so that the DL models from each GPU can be merged together. This synchronization and merging usually consists of an allreduce-style operation. This synchronization makes the effects of any imbalance between data loading and training more pronounced because, if even one worker and GPU become imbalanced, the performance of the entire distributed training will suffer.

## Deep Learning I/O Benchmark

Due to limited resources and due to this module being about a data movement service (DYAD), we do not need to actually train a DL model in this tutorial. Instead, we accurately show DYAD's benefit to DL training without performing the training itself by using the Argonne National Laboratory's [Deep Learning I/O benchmark](https://github.com/argonne-lcf/dlio_benchmark), or DLIO for short.

DLIO is a benchmark that aims to emulate the I/O behavior of Deep Learning applications. It has an extensible and modular design that allows it to use or mimic aspects (e.g., data formats, worker configuration, data loading-training balanced) of real-world applications. DLIO also has several useful support features, such as the ability to generate data with certain characteristics for users.

To learn more about DLIO, check out the following links:
* [DLIO Paper](https://ieeexplore.ieee.org/document/9499416)
* [DLIO Repo](https://github.com/argonne-lcf/dlio_benchmark)

## Accelerating DL training

As mentioned in the [Design of DYAD](#design-of-dyad) section, DYAD provides write-once, read-many file caching. This feature is extremely useful in read-heavy workloads, like distributed DL training.

In this section, we show DYAD's benefits to DL training using DLIO. More specifically, we first show an integration of DYAD into PyTorch through custom `Dataset` and `DataLoader` classes. Then, we run DYAD through a configuration of DLIO that mimics the training of a 3D U-Net model. Due to resource restrictions, we only run a small version of the 3D U-Net training pipeline. Finally, we show the I/O performance of DYAD compared against Lustre and [UnifyFS](https://ieeexplore.ieee.org/document/10177390) in training a full version of the 3D U-Net model at various scales on LLNL's [Corona](https://hpc.llnl.gov/hardware/compute-platforms/corona) supercomputer.

### Integrating DYAD into PyTorch

When using custom datasets or custom techniques/tools to read a dataset from storage, PyTorch requires the creation of `Dataset` and `DataLoader` classes. To use DYAD in PyTorch-based distributed DL training, we have implemented several of the `DYADTorchDataset` and `DyadTorchDataLoader` classes, which can both be found [here](../dlio_extensions/dyad_torch_data_loader.py). The `DYADTorchDataset` class is used to read samples from "remote" storage (if not previously read) or DYAD (if previously read), and it contains all the DYAD-specific code. The `DyadTorchDataLoader` class is a basic `DataLoader` which configures the "I/O" and "Map" steps of the data loading pipeline.

In the following code cells, we show the DYAD-specific code in `DYADTorchDataset`. As you will see, this code is very similar to standard Python file I/O. As a result, this code serves as an example of DYAD's transparency.

<div class="alert alert-block alert-info">
<b>Note:</b> due to several aspects of PyTorch's design (described below), DYAD cannot be used as transparently as normal. Normally, in Python, users would just have to replace the built-in `open` function for DYAD's `dyad_open`. As a result, this use case should be considered the *worst case* for DYAD's transparency.
</div>

In [ ]:
import os
import sys
import inspect
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML

sys.path.insert(0, os.path.abspath("../dlio_extensions/"))

from dyad_torch_data_loader import DYADTorchDataset

This first block of code shows the `DYADTorchDataset.worker_init` function. This function is called to initialize the I/O processes used to read samples. As a result, this function contains two parts: (1) the initialization of PyTorch internals and utilities and (2) the initialization of DYAD.

Normally, DYAD is configured using environment variables, and, as a result, DYAD's initialization can be hidden from users. However, due to PyTorch's complexity and challenges in correctly propagating environment variables through PyTorch's dynamic process spawning, DYAD's transparent, environment variable-based initialization cannot be used in `DYADTorchDataset`. Instead, we manually initialize and configure DYAD using `Dyad.init()`.

In [ ]:
display(HTML(highlight(inspect.getsource(DYADTorchDataset.worker_init), PythonLexer(), HtmlFormatter(full=True))))

This second block of code shows the `DYADTorchDataset.__getitem__` function. This function is called by `DyadTorchDataLoader` to read individual samples for a batch from disk. With other `Dataset` classes, this function would simply identify the file containing the requested sample and read that sample from remote storage (e.g., Lustre) using Python's built-in `open` function. On the other hand, `DYADTorchDataset` does four things. First, it identifies the file containing the requested sample. Second, it uses DYAD's `get_metadata` function to check if that file has already been cached into DYAD. Third, if the file has already been cached, it will retrieve the sample using DYAD's `dyad_open` function. This function retrieves the sample from a different node, if needed, and then makes that sample available through an interface equivalent to Python's built-in `open` function. Finally, if the file has **not** been cached, it will read the sample from remote storage (e.g., Lustre) and cache the sample into DYAD for more efficient future reading.

In [ ]:
display(HTML(highlight(inspect.getsource(DYADTorchDataset.__getitem__), PythonLexer(), HtmlFormatter(full=True))))

### Running DLIO with DYAD for a 3D U-Net model

Now that we have seen how DYAD is integrated into PyTorch, we configure and run DYAD through a configuration of DLIO that mimics the training of a 3D U-Net model.

#### Configuring DLIO and DYAD

First, we configure DYAD. DYAD requires three settings for configuration:
1. A namespace in the Flux key-value store, which DYAD will use for metadata management
2. A "managed directory," which DYAD will use to determine the files that should be tracked
3. A data transport layer (DTL) mode, which DYAD will use to select the underlying networking library for data transfer 

In [ ]:
kvs_namespace = "dyad"
managed_directory = "./dyad_data"
dtl_mode = "UCX" # We currently only support UCX, so do not change this

Next, we configure DLIO. DLIO requires several configuration settings. However, for this tutorial, the only one that should be set is the initial data directory, or the directory where the dataset initially resides at the start of training. When running DLIO, the `DYADTorchDataset` class dynamically copies files from this directory into DYAD's managed directory.

In [ ]:
initial_data_directory = "./dlio_data"

Finally, we set the remaining configurations for DLIO. These should not be edited because they depend on the directory structure and configuration of this tutorial.

In [ ]:
workers_per_node = 1
dyad_install_prefix = "/usr/local"
num_nodes = 2
dlio_extensions_dir = "/home/jovyan/flux-tutorial-2024/dlio_extensions"
workload = "dyad_unet3d_demo"

To properly set the environment variables needed for running DLIO with DYAD, we create an environment file that is compatible with the `--env-file` flag of `flux submit`.

In [ ]:
env_file = f"""
DYAD_KVS_NAMESPACE={kvs_namespace}
DYAD_DTL_MODE={dtl_mode}
DYAD_PATH={managed_directory}
LD_LIBRARY_PATH={dyad_install_prefix}/lib
PYTHONPATH={dlio_extensions_dir}:$PYTHONPATH
DLIO_PROFILER_ENABLE=0
"""

with open("dlio_env.txt", "w") as f:
    f.write(env_file)

#### Creating a Flux KVS namespace and starting the DYAD service

Next, we start the DYAD service. This involves two steps. First, we need to create a namespace withing the Flux key-value store. This namespace is used by DYAD to store metadata about cached files. This metadata is then used by DYAD's Hierarchical Data Locator to locate files.

In [ ]:
!flux kvs namespace create {kvs_namespace}

After creating the key-value store namespace, we start the DYAD service itself using the `flux module load` command. We run that command through `flux exec -r all` to deploy the service across all Flux brokers.

In [ ]:
!flux exec -r all flux module load {dyad_install_prefix}/lib/dyad.so --mode={dtl_mode} {managed_directory}

Finally, we check that the service and key-value store namespace were successfully created with the cells below.

In [ ]:
!flux module list

In [ ]:
!flux kvs namespace list

#### Generating data for the 3D U-Net

Before running DLIO, we need to obtain data for emulated training of the 3D U-Net. Instead of downloading the full dataset, we use DLIO to generate a smaller, synthetic version of the dataset for this tutorial.

In [ ]:
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {managed_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {managed_directory}/* 
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {initial_data_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {initial_data_directory}/* 

In [ ]:
!flux run -N {num_nodes} -o cpu-affinity=off --tasks-per-node={workers_per_node} --env-file=dlio_env.txt \
    dlio_benchmark --config-dir={dlio_extensions_dir}/configs workload={workload} \
        ++workload.dataset.data_folder={initial_data_directory} ++workload.workflow.generate_data=True \
        ++workload.workflow.train=False
!echo "FINISHED GENERATING DATA"

#### Emulating training of the 3D U-Net with DLIO

Now, we run DLIO using the command below. As DLIO runs, it prints out logging statements showing how long sample reading takes. At the end of the run, DLIO prints out a performance summary.

In [ ]:
!flux run -N {num_nodes} -o cpu-affinity=on --tasks-per-node={workers_per_node} --env-file=dlio_env.txt \
    dlio_benchmark --config-dir={dlio_extensions_dir}/configs workload={workload} \
        ++workload.dataset.data_folder={initial_data_directory} ++workload.workflow.generate_data=False \
        ++workload.workflow.train=True
!echo "FINISHED TRAINING"

#### Shutting down the DYAD service

Now that we are done running DLIO, we need to shutdown the DYAD service and remove the key-value store namespace used by DYAD. This is done with the two Flux commands below.

In [ ]:
!flux kvs namespace remove {kvs_namespace}
!flux exec -r all flux module remove dyad

The following cells show that the DYAD service has been removed and that the namespace has been removed from the key-value store.

In [ ]:
!flux module list

In [ ]:
!flux kvs namespace list

Finally, we need to remove all the files we generated while running DLIO. We use `flux run` to ensure that any node-local files are deleted.

In [ ]:
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {managed_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {managed_directory}/* 
!flux run -N {num_nodes} --tasks-per-node=1 mkdir -p {initial_data_directory} 
!flux run -N {num_nodes} --tasks-per-node=1 rm -r {initial_data_directory}/* 

### Evaluating DYAD's performance for the 3D U-Net at scale on Corona

<figure>
<img src="img/dyad-unet3d-results.svg">
<figcaption>
<i></i></figcaption>
</figure>

Figure X shows the performance of Lustre, [UnifyFS](https://ieeexplore.ieee.org/document/10177390), and DYAD in terms of runtime and I/O bandwidth for the full version of the 3D U-Net training. As explained on the [webpage for the KiTS19 Challenge](https://kits19.grand-challenge.org/), the dataset for the full version of this application consists of 10,240, NPZ-formatted image files, resulting in a total dataset size of 1.36 TB. Within each epoch of PyTorch-based training, the model processes batches of 4 images using 6 I/O processes per GPU. The model trains for 20 epochs without checkpointing. The model scales from 8 to 64 nodes of LLNL's [Corona](https://hpc.llnl.gov/hardware/compute-platforms/corona) supercomputer, with 8 GPUs per node.

In the leftmost plot of Figure X, we show the runtime of the training for Lustre, UnifyFS, and DYAD at 8, 16, 32, and 64 nodes. This plot shows that DYAD provides significant runtime improvement compared to Lustre and UnifyFS for the 3D U-Net, mainly due to locality optimizations. DYAD runs up to 7.5 times faster than Lustre and 1.88 times faster than UnifyFS, with less performance variability due to DYAD's use of node-local storage.

In the middle plot of Figure X, we show the bandwidth per epoch of training across 512 GPUs (64 nodes). Because DYAD's capabilities allow for on-the-fly caching of data, its performance starts similar to that of Lustre. As more data is cached into DYAD, its bandwidth increases to 140 GB/s due to DYAD's streaming RPC over RDMA protocol. Finally, as even more data is cached, DYAD's bandwidth reaches 1409 GB/s because DYAD's locality-aware caching allows almost all sample reads to be performed directly on node-local NVMe. In comparison, both Lustre and Unify maintain consistent bandwidths well under those of DYAD. By the 20th epoch, DYAD speeds up training by 10.62 times compared to UnifyFS.

Finally, in the rightmost plot of Figure X, we show how often DYAD retrieved data from node-local storage versus retrieving data from storage on a remote node in terms of percentage of data access requests. Initially, DYAD mostly performs remote requests. As training continues, more and more data is replicated with DYAD's locality-aware caching, resulting in a larger percentage of local requests. By epoch 13, almost all data is accessed through local requests. This transition from mostly remote requests to mostly local requests corresponds with the increase in bandwidth shown in the middle plot of Figure X.

# This concludes Module 4.

In this module, we covered:
1. Design of DYAD
2. Distributed DL training
3. Deep Learning I/O (DLIO) benchmark
4. Accelerating distributed DL training

To continue with the tutorial, open [Module 5](./05_flux_tutorial_conclusions.ipynb)